In [ ]:
import re

In [53]:
text = "Paul Newman was an American actor, but Paul Hollywood is a British TV Host. The name Paul is quite common."

In [ ]:
pattern = r"Paul [A-Z]\w+"
# Regex que busca por nomes que começam com "Paul" 
# seguido de um sobrenome que começa com letra maiúscula

In [41]:
matches = re.finditer(pattern, text)

In [42]:
for match in matches:
    print(match)

<re.Match object; span=(0, 11), match='Paul Newman'>
<re.Match object; span=(39, 53), match='Paul Hollywood'>


In [43]:
import spacy
from spacy.tokens import Span

In [44]:
nlp = spacy.blank("en")
doc = nlp(text)
original_ents = list(doc.ents)
mwt_ents = []
for match in re.finditer(pattern, doc.text):
    start, end = match.span()
    span = doc.char_span(start, end)
    print(span)
    if span is not None:
        mwt_ents.append((span.start, span.end, span.text))
for ent in mwt_ents:
    start, end, text = ent
    span = Span(doc, start, end, label="PERSON")
    original_ents.append(span)
doc.ents = original_ents
for ent in doc.ents:
    print(ent.text, ent.label_)

Paul Newman
Paul Hollywood
Paul Newman PERSON
Paul Hollywood PERSON


In [45]:
print(mwt_ents)

[(0, 2, 'Paul Newman'), (8, 10, 'Paul Hollywood')]


In [54]:
from spacy.language import Language

@Language.component("paul_ner")
def paul_ner(doc):
    pattern = r"Paul [A-Z]\w+" 
    original_ents = list(doc.ents)
    mwt_ents = []
    for match in re.finditer(pattern, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        print(span)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))
    for ent in mwt_ents:
        start, end, text = ent
        span = Span(doc, start, end, label="PERSON")
        original_ents.append(span)
    doc.ents = original_ents
    return(doc)

In [55]:
nlp2 = spacy.blank("en")
nlp2.add_pipe("paul_ner")

<function __main__.paul_ner(doc)>

In [57]:
doc2 = nlp2(text)
print(doc2.ents)

Paul Newman
Paul Hollywood
(Paul Newman, Paul Hollywood)


In [62]:
from spacy.language import Language
from spacy.util import filter_spans

@Language.component("holly_ner")
def holly_ner(doc):
    pattern = r"Hollywood" 
    original_ents = list(doc.ents)
    mwt_ents = []
    for match in re.finditer(pattern, doc.text):
        start, end = match.span()
        span = doc.char_span(start, end)
        print(span)
        if span is not None:
            mwt_ents.append((span.start, span.end, span.text))
    for ent in mwt_ents:
        start, end, text = ent
        span = Span(doc, start, end, label="CINEMA")
        original_ents.append(span)
    filtered = filter_spans(original_ents)
    doc.ents = filtered
    return(doc)

In [63]:
nlp3 = spacy.load("en_core_web_sm")
nlp3.add_pipe("holly_ner")

<function __main__.holly_ner(doc)>

In [68]:
doc3 = nlp3(text)
for ent in doc3.ents:
    print(ent.text, ent.label_)

Hollywood
Paul Newman PERSON
American NORP
Paul Hollywood PERSON
British NORP
Paul PERSON
